In [1]:
import pandas as pd
import featuretools as ft
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold, cross_val_score
import numpy as np
from tpot import TPOTClassifier
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import balanced_accuracy_score
import joblib  # 用于保存交互项模式和模型

In [2]:
X_y_group_train = pd.read_csv('mid_data/X_y_group_train_updated_v8.7_线性回归v7&岭回归v2.csv')

print("Adding numeric labels y")
le = LabelEncoder()
X_y_group_train["y"] = le.fit_transform(X_y_group_train["label"])
# reordering columns:
X_y_group_train = X_y_group_train[["dataset", "variable"] + X_y_group_train.columns.drop(["dataset", "variable", "label", "y"]).tolist() + ["label", "y"]]


blacklist = ["ttest(v,X)", "pvalue(ttest(v,X))<=0.05", "ttest(v,Y)", "pvalue(ttest(v,Y))<=0.05", "ttest(X,Y)", "pvalue(ttest(X,Y))<=0.05"]
columns_to_drop = [col for col in blacklist if col in X_y_group_train.columns]
X_y_group_train = X_y_group_train.drop(columns=columns_to_drop)

numeric_columns = X_y_group_train.select_dtypes(include=[np.number]).columns
X_y_group_train[numeric_columns] = X_y_group_train[numeric_columns].fillna(X_y_group_train[numeric_columns].mean())

display(X_y_group_train)

print("Extracting X_train, y_train, and group")
X_train = X_y_group_train.drop(["variable", "dataset", "label", "y"], axis="columns")

y_train = X_y_group_train["y"]

Adding numeric labels y


,dataset,variable,dimension,"corr(v,X)","corr(v,Y)","max(corr(v, others))","min(corr(v, others))","mean(corr(v, others))","std(corr(v, others))","corr(X,Y)",...,v~X_coefficient,v_squared~X_coefficient,v_cos~X_coefficient,v_sin~X_coefficient,X~Y_coefficient,v~Y_ridge_coefficient,v~X_ridge_coefficient,X~Y_ridge_coefficient,label,y
0,0,0,8,0.169735,-0.113595,0.791467,0.027355,0.282376,0.298969,-0.771058,...,2.581106,0.642049,0.627756,-1.853991,-0.243933,-0.270012,0.746844,-0.210404,Consequence of Y,5
1,0,2,8,-0.072334,0.139419,0.230139,0.003233,0.090511,0.082201,-0.771058,...,-0.057149,-0.355636,-0.302878,0.029001,-0.243933,-0.004777,-0.015475,-0.210404,Independent,6
2,0,3,8,0.123115,-0.238769,0.781051,0.012225,0.257037,0.288165,-0.771058,...,0.043431,-0.408441,-0.632885,-0.423769,-0.243933,0.087481,-0.235369,-0.210404,Cause of Y,1
3,0,4,8,-0.001935,0.013921,0.147408,0.001935,0.052451,0.053457,-0.771058,...,-0.055143,0.381220,0.427163,0.116956,-0.243933,-0.016076,0.045268,-0.210404,Cause of Y,1
4,0,5,8,0.284323,-0.466570,0.791467,0.064815,0.426306,0.250048,-0.771058,...,3.443463,-0.992699,-0.797171,-2.181048,-0.243933,-0.454817,1.085383,-0.210404,Mediator,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142905,46997,6,7,0.014740,0.697089,0.748635,0.013858,0.376630,0.352655,-0.089641,...,-0.414477,2.767481,2.688398,0.383910,-0.048813,0.046537,0.000815,-0.049793,Cause of Y,1
142906,46997,7,7,-0.007397,-0.116481,0.998843,0.007397,0.180741,0.333308,-0.089641,...,3.145600,1.175395,-5.843335,-3.135427,-0.048813,-0.015032,-0.000532,-0.049793,Cause of Y,1
142907,46997,8,7,-0.011658,0.908029,0.970364,0.011658,0.469777,0.441105,-0.089641,...,0.553463,2.599347,2.702726,-0.380755,-0.048813,0.499969,-0.000334,-0.049793,Cause of Y,1
142908,46998,0,2,0.083546,-0.019665,0.083546,0.016856,0.040022,0.037719,0.036862,...,0.449761,0.398762,0.421113,-0.405605,0.109254,-0.010355,0.044703,0.120088,Consequence of X,4


Extracting X_train, y_train, and group


In [9]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, balanced_accuracy_score

# 检查类别数量
num_classes = y_train.nunique()
print('num_classes', num_classes)
# 将数据拆分为训练集和验证集
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42)

# 初始化XGBClassifier，使用GPU加速，选择合适的超参数
xgb_clf = XGBClassifier(
    objective='multi:softprob',
    num_class=num_classes,
    eval_metric='mlogloss',
    use_label_encoder=False,
    tree_method='gpu_hist',  # 使用GPU加速
    n_estimators=500,
    max_depth=8,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=0.5,
    verbosity=1  # 设置为1，输出必要的训练信息
)

# 开始训练
print("Starting training...")
xgb_clf.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=True)

# 在验证集上评估模型
y_pred = xgb_clf.predict(X_val)
balanced_acc = balanced_accuracy_score(y_val, y_pred)
print(f"Validation Balanced Accuracy: {balanced_acc:.4f}")
print("Classification Report:")
print(classification_report(y_val, y_pred))


num_classes 8
Starting training...


e:\Softwares\Anaconda\CondaEnvs\casual\lib\site-packages\xgboost\core.py:158: UserWarning: [22:50:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
e:\Softwares\Anaconda\CondaEnvs\casual\lib\site-packages\xgboost\core.py:158: UserWarning: [22:50:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-mlogloss:1.99383
[1]	validation_0-mlogloss:1.92045
[2]	validation_0-mlogloss:1.85680
[3]	validation_0-mlogloss:1.80156
[4]	validation_0-mlogloss:1.75217
[5]	validation_0-mlogloss:1.70531
[6]	validation_0-mlogloss:1.66351
[7]	validation_0-mlogloss:1.62594
[8]	validation_0-mlogloss:1.58973
[9]	validation_0-mlogloss:1.55552
[10]	validation_0-mlogloss:1.52346
[11]	validation_0-mlogloss:1.49377
[12]	validation_0-mlogloss:1.46547
[13]	validation_0-mlogloss:1.43988
[14]	validation_0-mlogloss:1.41509
[15]	validation_0-mlogloss:1.39172
[16]	validation_0-mlogloss:1.36976
[17]	validation_0-mlogloss:1.34957
[18]	validation_0-mlogloss:1.33019
[19]	validation_0-mlogloss:1.31227
[20]	validation_0-mlogloss:1.29512
[21]	validation_0-mlogloss:1.27884
[22]	validation_0-mlogloss:1.26419
[23]	validation_0-mlogloss:1.24896
[24]	validation_0-mlogloss:1.23522
[25]	validation_0-mlogloss:1.22142
[26]	validation_0-mlogloss:1.20865
[27]	validation_0-mlogloss:1.19661
[28]	validation_0-mlogloss:1.1

e:\Softwares\Anaconda\CondaEnvs\casual\lib\site-packages\xgboost\core.py:158: UserWarning: [22:51:35] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Validation Balanced Accuracy: 0.5141
Classification Report:
              precision    recall  f1-score   support

           0       0.59      0.47      0.52      2026
           1       0.77      0.75      0.76      4289
           2       0.43      0.24      0.31       845
           3       0.48      0.44      0.46      1454
           4       0.59      0.63      0.61      3107
           5       0.58      0.48      0.52      1360
           6       0.78      0.91      0.84      8808
           7       0.35      0.19      0.24       977

    accuracy                           0.69     22866
   macro avg       0.57      0.51      0.53     22866
weighted avg       0.67      0.69      0.68     22866



In [ ]:
## Best:
xgb_clf = XGBClassifier(
    objective='multi:softprob',
    num_class=num_classes,
    eval_metric='mlogloss',
    use_label_encoder=False,
    tree_method='gpu_hist',  # 使用GPU加速
    n_estimators=500,
    max_depth=8,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=0.5,
    verbosity=1  # 设置为1，输出必要的训练信息
)

#0.5141

In [7]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from itertools import product
import numpy as np

# 创建训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

def grid_search_xgb(X_train, y_train, param_grid):
    # 将训练集进一步拆分为训练集和验证集
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42)
    
    num_classes = y_train.nunique()
    best_score = 0
    best_params = {}
    
    # 生成所有参数组合
    param_combinations = list(product(*param_grid.values()))
    total_combinations = len(param_combinations)
    
    for i, params in enumerate(param_combinations, 1):
        current_params = dict(zip(param_grid.keys(), params))
        print(f"Testing combination {i}/{total_combinations}: {current_params}")
        
        xgb_clf = XGBClassifier(
            objective='multi:softprob',
            num_class=num_classes,
            eval_metric='mlogloss',
            use_label_encoder=False,
            tree_method='gpu_hist',
            verbosity=0,
            **current_params
        )
        
        xgb_clf.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)
        
        y_pred = xgb_clf.predict(X_val)
        score = balanced_accuracy_score(y_val, y_pred)
        
        print(f"Validation Balanced Accuracy: {score:.4f}")
        
        if score > best_score:
            best_score = score
            best_params = current_params
            print(f"New best score: {best_score:.4f}")
        
        print("--------------------")
    
    return best_params, best_score

# 定义参数网格
param_grid = {
    'n_estimators': [300, 500, 800],
    'max_depth': [5, 7, 9],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'reg_alpha': [0.1, 0.5, 1],
    'reg_lambda': [0.1, 0.5, 1]
}

# 执行网格搜索
best_params, best_score = grid_search_xgb(X_train, y_train, param_grid)

print("Best parameters:", best_params)
print(f"Best Validation Balanced Accuracy: {best_score:.4f}")

# 使用最佳参数训练最终模型
final_model = XGBClassifier(
    objective='multi:softprob',
    num_class=y_train.nunique(),
    eval_metric='mlogloss',
    use_label_encoder=False,
    tree_method='gpu_hist',
    verbosity=1,
    **best_params
)

print("Training final model with best parameters...")
final_model.fit(X_train, y_train, verbose=True)

# 在测试集上评估最终模型
y_pred = final_model.predict(X_test)
final_score = balanced_accuracy_score(y_test, y_pred)
print(f"Final Test Balanced Accuracy: {final_score:.4f}")

Testing combination 1/2187: {'n_estimators': 300, 'max_depth': 5, 'learning_rate': 0.01, 'subsample': 0.7, 'colsample_bytree': 0.7, 'reg_alpha': 0.1, 'reg_lambda': 0.1}
Validation Balanced Accuracy: 0.4304
New best score: 0.4304
--------------------
Testing combination 2/2187: {'n_estimators': 300, 'max_depth': 5, 'learning_rate': 0.01, 'subsample': 0.7, 'colsample_bytree': 0.7, 'reg_alpha': 0.1, 'reg_lambda': 0.5}
Validation Balanced Accuracy: 0.4306
New best score: 0.4306
--------------------
Testing combination 3/2187: {'n_estimators': 300, 'max_depth': 5, 'learning_rate': 0.01, 'subsample': 0.7, 'colsample_bytree': 0.7, 'reg_alpha': 0.1, 'reg_lambda': 1}
Validation Balanced Accuracy: 0.4303
--------------------
Testing combination 4/2187: {'n_estimators': 300, 'max_depth': 5, 'learning_rate': 0.01, 'subsample': 0.7, 'colsample_bytree': 0.7, 'reg_alpha': 0.5, 'reg_lambda': 0.1}
Validation Balanced Accuracy: 0.4303
--------------------
Testing combination 5/2187: {'n_estimators': 300

KeyboardInterrupt: 